In [1]:
import ctypes
import sys
import os 
import numpy as np
from ctypes import POINTER
import matplotlib.pyplot as plt
from mc_for_bimetric import calc_likelihood


In [2]:
cwd =  os.getcwd()
lib_h = ctypes.CDLL(cwd + "/friedmann_Cs_lib.so") 

In [3]:
def load_data(fname="data.txt"):
    f = np.genfromtxt(fname,delimiter="\t")
    z = np.squeeze(f[:,0])
    fs8_data = np.squeeze(f[:,1])
    fs8_sig2 = np.square(np.squeeze(f[:,2]))
    om_fid = np.squeeze(f[:,3])
    
    c = np.diag(fs8_sig2)
    c[5,5] = 6.4000*0.001
    c[4,4] = 3.969*0.001
    c[3,3] = 5.184*0.001
    c[4,5] = 2.570*0.001
    c[3,5] = 0.000
    c[5,4] = 2.570*0.001
    c[3,4] = 2.540*0.001
    c[5,3] = 0.000*0.001
    c[4,3] = 2.540*0.001
    
    c_i = np.linalg.inv(c)
    rat_den = np.sqrt(om_fid*np.power(1.0+z,3.0)+(1.0-om_fid))
    mz = -1.0*z
    sa = mz.argsort()
    sa_inv = mz.argsort().argsort()
    
    cn = np.array([1,1,2,1,2,1,1,1,1,1,1,1,1,1,1,3,3,3])
    
    return z,fs8_data,c_i,rat_den,sa,sa_inv,cn
    
    

In [4]:
z,f,ci,r,s,si,cn = load_data()
n=18
print(cn.dtype)
cnw = ctypes.c_long*(n)
u = np.ctypeslib.as_ctypes(cn)

int64


In [5]:
u

In [ ]:
l = calc_likelihood(z,f,r,ci,cn,s,si,0.4,0.0,0.79)

In [ ]:
l